Name: **Siddhesh Dhainje**<br>
Div: **BE10-Q10**<br>
Roll no: **43221**<br>
Title: **Assignment 5: Implement the Continuous Bag of Words (CBOW) Model**<br>

In [1]:
#importing libraries
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.utils import pad_sequences
import numpy as np
import pandas as pd

In [5]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()
dl_data

['Deep',
 'learning',
 '(also',
 'known',
 'as',
 'deep',
 'structured',
 'learning)',
 'is',
 'part',
 'of',
 'a',
 'broader',
 'family',
 'of',
 'machine',
 'learning',
 'methods',
 'based',
 'on',
 'artificial',
 'neural',
 'networks',
 'with',
 'representation',
 'learning.',
 'Learning',
 'can',
 'be',
 'supervised,',
 'semi-supervised',
 'or',
 'unsupervised.',
 'Deep-learning',
 'architectures',
 'such',
 'as',
 'deep',
 'neural',
 'networks,',
 'deep',
 'belief',
 'networks,',
 'deep',
 'reinforcement',
 'learning,',
 'recurrent',
 'neural',
 'networks,',
 'convolutional',
 'neural',
 'networks',
 'and',
 'Transformers',
 'have',
 'been',
 'applied',
 'to',
 'fields',
 'including',
 'computer',
 'vision,',
 'speech',
 'recognition,',
 'natural',
 'language',
 'processing,',
 'machine',
 'translation,',
 'bioinformatics,',
 'drug',
 'design,',
 'medical',
 'image',
 'analysis,',
 'climate',
 'science,',
 'material',
 'inspection',
 'and',
 'board',
 'game',
 'programs,',
 'where

In [6]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [4]:
#generating (context word, target/label word) pairs
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield (x, y)
            
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [5]:
#model building
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15,075
Trainable params: 15,075
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Epoch: 1 	Loss: 433.9979591369629

Epoch: 2 	Loss: 429.12920451164246

Epoch: 3 	Loss: 425.6162567138672

Epoch: 4 	Loss: 422.305300951004

Epoch: 5 	Loss: 419.82955026626587



In [7]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,-0.051325,0.015496,-0.001289,-0.000091,0.031098,-0.004511,0.048802,0.039863,0.038735,-0.011285,...,-0.021157,-0.006114,0.014426,-0.025363,-0.001656,-0.007467,0.010676,0.054003,-0.010888,0.013069
networks,0.030027,-0.013018,0.014034,0.053244,0.000094,0.018606,0.017825,0.043783,0.039989,0.022160,...,-0.034817,-0.050645,0.042752,-0.028074,-0.014875,-0.012554,0.018793,0.038520,-0.050688,0.029010
neural,-0.000493,-0.039977,0.020773,-0.019352,0.002148,-0.039686,-0.046782,0.023946,0.028829,0.034786,...,0.025748,0.018518,0.013404,-0.031155,-0.013245,-0.038053,-0.032669,-0.012383,-0.042692,-0.008998
and,0.033945,0.031775,0.040324,0.020284,-0.041136,-0.032880,0.014349,0.003366,0.024911,-0.039747,...,0.002081,-0.030681,0.049420,-0.001499,-0.031708,0.012737,-0.033358,0.027951,0.019079,-0.031530
as,-0.023164,0.021082,-0.043694,-0.016592,0.029788,-0.039470,-0.044566,-0.049052,-0.000571,-0.018558,...,-0.025940,-0.036442,-0.033984,-0.038409,-0.008912,0.045914,0.047436,0.012006,-0.027358,0.006428


In [8]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['design', 'medical', 'fields', 'image', 'transformers']}